<a href="https://colab.research.google.com/github/diwakar-vsingh/swift-models/blob/DeepLab/DeepLab/S4TFdeepLabV3%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Program

### Import Standard Swift Libraries

In [ ]:
import TensorFlow
import Python
import Foundation
import Path
import Just

2020-11-28 02:05:03.555136: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


### Import Python Libraries

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

let plt = Python.import("matplotlib.pyplot")
let np  = Python.import("numpy")
let subprocess = Python.import("subprocess")
let glob = Python.import("glob")
let pil = Python.import("PIL")
let tf = Python.import("tensorflow")
let h5py = Python.import("h5py")
let os = Python.import("os")
let shutil = Python.import("shutil")

## Import Model Weights

In [ ]:
let modelName = "deeplabv3plus"
let backbone = "xception"

struct Weights {
    let name: String
    let layer: Tensor<Float>
}

func get_pretrained_weights_dict(modelName: String, backbone: String) -> Array<Weights> {
  var f = Python.None
  let weightFileName = "\(modelName)_\(backbone).npz"
  let weights = np.load(weightFileName)

  var weights_array = Array<Weights>()
  for param in weights {
      weights_array.append(Weights(name: String(param)!, layer: Tensor<Float>(numpy: weights[param])!))
  }
  return weights_array
}

var weightsDict = get_pretrained_weights_dict(modelName: model_name, backbone: backbone)

In [ ]:
// shutil.rmtree("planet_tiny/")

Use `print()` to show values.


In [ ]:
let command = "wget -nv -O- http://download.tensorflow.org/models/deeplabv3_pascal_trainval_2018_01_04.tar.gz | tar xzf - -C ."
subprocess.call(command, shell: true)

2020-11-26 21:18:11 URL:http://download.tensorflow.org/models/deeplabv3_pascal_trainval_2018_01_04.tar.gz [460669898/460669898] -> "-" [1]


Use `print()` to show values.


## Configure Model

### Modified DeptwiseConv2D as DepthConv2D 

In [ ]:
import _Differentiation

@differentiable(wrt: (input, filter))
public func depthConv2D<Scalar: TensorFlowFloatingPoint>(
  _ input: Tensor<Scalar>,
  filter: Tensor<Scalar>,
  strides: (Int, Int, Int, Int),
  padding: Padding,
  dilations: (Int, Int, Int, Int)
) -> Tensor<Scalar> {
  precondition(input.shape.rank == 4, "The input must have rank 4.")
  precondition(filter.shape.rank == 4, "The filter must have rank 4.")
  return _Raw.depthwiseConv2dNative(
    input,
    filter: filter,
    strides: [Int32(strides.0), Int32(strides.1), Int32(strides.2), Int32(strides.3)],
    padding: padding.raw,
    dilations: [Int32(dilations.0), Int32(dilations.1), Int32(dilations.2), Int32(dilations.3)])
}

@usableFromInline
@derivative(of: depthConv2D)
func _vjpDepthConv2D<Scalar: TensorFlowFloatingPoint>(
  _ x: Tensor<Scalar>,
  filter: Tensor<Scalar>,
  strides: (Int, Int, Int, Int),
  padding: Padding,
  dilations: (Int, Int, Int, Int)
) -> (value: Tensor<Scalar>, pullback: (Tensor<Scalar>) -> (Tensor<Scalar>, Tensor<Scalar>)) {
  let value = depthConv2D(
    x, filter: filter, strides: strides,
    padding: padding, dilations: dilations)
  return (
    value,
    { v in
      (
        depthConv2dBackpropInput(
          v, shape: x.shapeTensor, filter: filter,
          strides: strides, padding: padding, dilations: dilations),
        depthConv2dBackpropFilter(
          v, input: x, filterSizes: filter.shapeTensor,
          strides: strides, padding: padding, dilations: dilations)
      )
    }
  )
}

/// TensorFlow builtin depthwiseConv2D gradient helper for the input.
@differentiable(wrt: (x, filter))
@usableFromInline
func depthConv2dBackpropInput<Scalar: TensorFlowFloatingPoint>(
  _ x: Tensor<Scalar>,
  shape: Tensor<Int32>,
  filter: Tensor<Scalar>,
  strides: (Int, Int, Int, Int),
  padding: Padding,
  dilations: (Int, Int, Int, Int)
) -> Tensor<Scalar> {
  return _Raw.depthwiseConv2dNativeBackpropInput(
    inputSizes: shape,
    filter: filter,
    outBackprop: x,
    strides: [Int32(strides.0), Int32(strides.1), Int32(strides.2), Int32(strides.3)],
    padding: padding.raw,
    dilations: [Int32(dilations.0), Int32(dilations.1), Int32(dilations.2), Int32(dilations.3)])
}

@usableFromInline
@derivative(of: depthConv2dBackpropInput)
func _vjpDepthConv2dBackpropInput<Scalar: TensorFlowFloatingPoint>(
  _ x: Tensor<Scalar>,
  _ shape: Tensor<Int32>,
  _ filter: Tensor<Scalar>,
  _ strides: (Int, Int, Int, Int),
  _ padding: Padding,
  _ dilations: (Int, Int, Int, Int)
) -> (value: Tensor<Scalar>, pullback: (Tensor<Scalar>) -> (Tensor<Scalar>, Tensor<Scalar>)) {
  let value = depthConv2dBackpropInput(
    x, shape: shape, filter: filter, strides: strides,
    padding: padding, dilations: dilations)
  return (
    value,
    { v in
      (
        depthConv2D(v, filter: filter, strides: strides, padding: padding, dilations: dilations),
        depthConv2dBackpropFilter(
          x, input: v, filterSizes: filter.shapeTensor,
          strides: strides, padding: padding, dilations: dilations)
      )

    }
  )
}

/// TensorFlow builtin depthwiseConv2D gradient helper for the filter.
@differentiable(wrt: (x, input))
@usableFromInline
func depthConv2dBackpropFilter<Scalar: TensorFlowFloatingPoint>(
  _ x: Tensor<Scalar>,
  input: Tensor<Scalar>,
  filterSizes: Tensor<Int32>,
  strides: (Int, Int, Int, Int),
  padding: Padding,
  dilations: (Int, Int, Int, Int)
) -> Tensor<Scalar> {
  return _Raw.depthwiseConv2dNativeBackpropFilter(
    input,
    filterSizes: filterSizes,
    outBackprop: x,
    strides: [Int32(strides.0), Int32(strides.1), Int32(strides.2), Int32(strides.3)],
    padding: padding.raw,
    dilations: [Int32(dilations.0), Int32(dilations.1), Int32(dilations.2), Int32(dilations.3)])
}

@usableFromInline
@derivative(of: depthConv2dBackpropFilter)
func _vjpDepthConv2dBackpropFilter<Scalar: TensorFlowFloatingPoint>(
  _ x: Tensor<Scalar>,
  _ input: Tensor<Scalar>,
  _ filterSizes: Tensor<Int32>,
  _ strides: (Int, Int, Int, Int),
  _ padding: Padding,
  _ dilations: (Int, Int, Int, Int)
) -> (value: Tensor<Scalar>, pullback: (Tensor<Scalar>) -> (Tensor<Scalar>, Tensor<Scalar>)) {
  let value = depthConv2dBackpropFilter(
    x, input: input, filterSizes: filterSizes,
    strides: strides, padding: padding, dilations: dilations)
  return (
    value,
    { v in
      (
        depthConv2D(input, filter: v, strides: strides, padding: padding, dilations: dilations),
        depthConv2dBackpropInput(
          x, shape: x.shapeTensor, filter: v, strides: strides,
          padding: padding, dilations: dilations)
      )
    }
  )
}

In [ ]:
@frozen
public struct DepthConv2D<Scalar: TensorFlowFloatingPoint>: Layer {
  /// The 4-D convolution kernel.
  public var filter: Tensor<Scalar>
  /// The bias vector.
  public var bias: Tensor<Scalar>
  /// The element-wise activation function.
  @noDerivative public let activation: Activation
  /// The strides of the sliding window for spatial dimensions.
  @noDerivative public let strides: (Int, Int)
  /// The padding algorithm for convolution.
  @noDerivative public let padding: Padding
  /// The dilation factor for spatial dimensions.
  @noDerivative public let dilations: (Int, Int)
  /// Note: `useBias` is a workaround for TF-1153: optional differentiation support.
  @noDerivative private let useBias: Bool

  /// The element-wise activation function type.
  public typealias Activation = @differentiable (Tensor<Scalar>) -> Tensor<Scalar>

  /// Creates a `DepthwiseConv2D` layer with the specified filter, bias, activation function,
  /// strides, and padding.
  ///
  /// - Parameters:
  ///   - filter: The 4-D convolution kernel.
  ///   - bias: The bias vector.
  ///   - activation: The element-wise activation function.
  ///   - strides: The strides of the sliding window for spatial dimensions.
  ///   - padding: The padding algorithm for convolution.
  ///   - dilations: The dilation factors for spatial dimensions.
  
  public init(
    filter: Tensor<Scalar>,
    bias: Tensor<Scalar>? = nil,
    activation: @escaping Activation = identity,
    strides: (Int, Int) = (1, 1),
    padding: Padding = .valid,
    dilations: (Int, Int) = (1, 1)
  ) {
    self.filter = filter
    self.bias = bias ?? .zero
    self.activation = activation
    self.strides = strides
    self.padding = padding
    self.dilations = dilations
    useBias = (bias != nil)
  }

  /// Returns the output obtained from applying the layer to the given input.
  ///
  /// - Parameter input: The input to the layer of shape,
  ///   [batch count, input height, input width, input channel count]
  /// - Returns: The output of shape,
  ///   [batch count, output height, output width, input channel count * channel multiplier]
  @differentiable
  public func forward(_ input: Tensor<Scalar>) -> Tensor<Scalar> {
    let conv = depthConv2D(
      input,
      filter: filter,
      strides: (1, strides.0, strides.1, 1),
      padding: padding,
      dilations: (1, dilations.0, dilations.1, 1))
    return activation(useBias ? (conv + bias) : conv)
  }
}

extension DepthConv2D {
  /// Creates a `DepthwiseConv2D` layer with the specified filter shape, strides, padding, and
  /// element-wise activation function.
  ///
  /// - Parameters:
  ///   - filterShape: The shape of the 4-D convolution kernel with form,
  ///     [filter width, filter height, input channel count, channel multiplier].
  ///   - strides: The strides of the sliding window for spatial/spatio-temporal dimensions.
  ///   - padding: The padding algorithm for convolution.
  ///   - dilations: The dilation factors for spatial dimensions.
  ///   - activation: The element-wise activation function.
  ///   - filterInitializer: Initializer to use for the filter parameters.
  ///   - biasInitializer: Initializer to use for the bias parameters.
  public init(
    filterShape: (Int, Int, Int, Int),
    strides: (Int, Int) = (1, 1),
    padding: Padding = .valid,
    dilations: (Int, Int) = (1, 1),
    activation: @escaping Activation = identity,
    useBias: Bool = true,
    filterInitializer: ParameterInitializer<Scalar> = glorotUniform(),
    biasInitializer: ParameterInitializer<Scalar> = zeros()
  ) {
    let filterTensorShape = TensorShape([
      filterShape.0, filterShape.1, filterShape.2, filterShape.3,
    ])
    self.init(
      filter: filterInitializer(filterTensorShape),
      bias: useBias ? biasInitializer([filterShape.2 * filterShape.3]) : nil,
      activation: activation,
      strides: strides,
      padding: padding,
      dilations: dilations)
  }
}

In [ ]:
let input = Tensor<Float>(ones: [1, 256, 256, 64])
let dconv = DepthConv2D<Float>(filterShape: (3, 3, 64, 1))
let output = dconv(input)
print(output.shape)

[1, 254, 254, 64]


### Xception Modules

#### ConvBlock

In [ ]:
public struct ConvBlock: Layer {
  @noDerivative public var applyBatchNorm: Bool
  @noDerivative public var depthAct: Bool
  @noDerivative public var applyDropout: Bool

  public var conv: Conv2D<Float>
  public var batchNorm: BatchNorm<Float>
  public var dropout: Dropout<Float>

  public init(
    filterShape: (Int, Int, Int, Int),
    strides: (Int, Int) = (1,1),
    useBias: Bool = false,
    epsilon: Float = 1e-3,
    applyBatchNorm: Bool = true,
    depthAct: Bool = true,
    applyDropout: Bool = false,
    probability: Double = 0.1
  ){
    self.applyBatchNorm = applyBatchNorm
    self.depthAct = depthAct
    self.applyDropout = applyDropout
    
    conv = Conv2D<Float>(
      filterShape: filterShape, 
      strides: strides, 
      padding: .same, 
      useBias: useBias)
    batchNorm = BatchNorm<Float>(featureCount: filterShape.3, epsilon: epsilon)
    dropout = Dropout<Float>(probability: probability)
  }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float>{
    var convolved: Tensor<Float>
    if self.applyBatchNorm {
      convolved = input.sequenced(through: conv, batchNorm)
    } else {
      convolved = conv(input)
    }
    if self.depthAct {
      convolved = relu(convolved)
    } 
    if self.applyDropout {
      convolved = dropout(convolved)
    } 
    return convolved
  }
}

#### DepthwiseConvBlock

In [ ]:
public struct DepthwiseConvBlock: Layer {
  @noDerivative let strides: (Int, Int)
  @noDerivative public var depthAct: Bool

  public var dConv: DepthConv2D<Float>
  public var batchNorm1: BatchNorm<Float>
  public var conv: Conv2D<Float>
  public var batchNorm2: BatchNorm<Float>

  public init(
    filterShape: (Int, Int, Int, Int),
    strides: (Int, Int) = (1,1),  
    dilations: Int = 1, 
    epsilon: Float = 1e-3,
    depthAct: Bool = false
    ) {
      self.strides = strides
      self.depthAct = depthAct

      dConv = DepthConv2D<Float>(
        filterShape: (3, 3, filterShape.2, 1),
        strides: strides,
        padding: .same,
        dilations: (dilations, dilations),
        useBias: false)
      batchNorm1 = BatchNorm<Float>(featureCount: filterShape.2, epsilon: epsilon)
    
      conv = Conv2D<Float>(
        filterShape: (1, 1, filterShape.2, filterShape.3),
        strides: (1,1),
        padding: .same,
        useBias: false)
      batchNorm2 = BatchNorm<Float>(featureCount: filterShape.3, epsilon: epsilon)
      }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
    var convolved1 = input
    if !self.depthAct {
      convolved1 = relu(input)
    }
    convolved1 = convolved1.sequenced(through: dConv, batchNorm1)

    if self.depthAct {convolved1 = relu(convolved1)}
    var convolved2 = convolved1.sequenced(through: conv, batchNorm2)

    if self.depthAct {convolved2 = relu(convolved2)}
    return convolved2
  }
}

In [ ]:
let input = Tensor<Float> (ones: [1, 256, 256, 64])
let conv = DepthwiseConvBlock(filterShape: (3, 3, 64, 128), dilations: 2)
let conv2 = DepthwiseConvBlock(filterShape: (3, 3, 728, 728)), dilations: dilationRates.0)
let temp = conv(input)
print(temp.shape)

[1, 256, 256, 128]


In [ ]:
public struct MiddleFlow: Layer {
  public var middleBlock: [DepthwiseConvBlock] = []

  public init() {
    middleBlock.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 728)))
    middleBlock.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 728)))
    middleBlock.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 728)))
  }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
    return middleBlock.differentiableReduce(input) {$1($0)}
  }
}

In [ ]:
let input = Tensor<Float> (ones: [1, 32, 32, 728])
let conv = MiddleFlow()
let temp = conv(input)
print(temp.shape)

[1, 32, 32, 728]


#### ASPP Block

In [ ]:
public struct ASPPLayer: Layer {
  
  public var dConv: DepthConv2D<Float>
  public var batchNorm1: BatchNorm<Float>
  public var conv: Conv2D<Float>
  public var batchNorm2: BatchNorm<Float>

  public init(
    filterShape: (Int, Int, Int, Int),
    dilations: Int = 1,
    epsilon: Float = 1e-5
  ){
    dConv = DepthConv2D<Float>(
      filterShape: (3, 3, filterShape.2, 1),
      strides: (1,1),
      padding: .same,
      dilations: (dilations, dilations),
      useBias: false)
    batchNorm1 = BatchNorm<Float>(featureCount: filterShape.2, epsilon: epsilon)
    
    conv = Conv2D<Float>(
        filterShape: (1, 1, filterShape.2, filterShape.3),
        strides: (1,1),
        padding: .same,
        useBias: false)
    batchNorm2 = BatchNorm<Float>(featureCount: filterShape.3, epsilon: epsilon)
  }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float>{
    var convolved = relu(input.sequenced(through: dConv, batchNorm1))
    convolved = relu(convolved.sequenced(through: conv, batchNorm2))
    return convolved
  }
}

In [ ]:
let input = Tensor<Float>(ones: [1, 32, 32, 2048])
let aspp = ASPPLayer(filterShape: (3, 3, 2048, 256))
let out = aspp(input)
print(out.shape)

[1, 32, 32, 256]


#### ImageFetures Block

In [ ]:
public struct ImageFeature: Layer {

  public var conv: Conv2D<Float>
  public var batchNorm: BatchNorm<Float>

  public init(
    filterShape: (Int, Int, Int, Int),
    epsilon: Float = 1e-5
  ) {
    conv = Conv2D<Float>(
      filterShape: (filterShape.0, filterShape.1, filterShape.2, filterShape.3),
      strides: (1,1),
      padding: .same,
      useBias: false)
    batchNorm = BatchNorm<Float>(featureCount: filterShape.3, epsilon: epsilon)
  }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float>{
    let hw = input.shape[1...2]
    var avgPooling = AvgPool2D<Float>(poolSize: (hw[0], hw[1]), strides: (hw[0], hw[1]))
    
    return resize(
      images: relu(input.sequenced(through: avgPooling, conv, batchNorm)),
      size: (newHeight: hw[0], newWidth: hw[1]),
      method: .bilinear)
  }
}

In [ ]:
let input = Tensor<Float>(ones: [1, 32, 32, 2048])
let aspp = ImageFeature(filterShape: (1, 1, 2048, 256))
let out = aspp(input)
print(out.shape)

[1, 32, 32, 256]


### DeepLabV3+ Model

In [ ]:
let dilationRates = (1,2)
let atrousRates = (6, 12, 18)
let inputSize = 512
let imageSize = (inputSize, inputSize, 3)
let classes = 21
let weight = "pascalVOC"

In [ ]:
let input1 = Tensor<Float> (ones: [1, 32, 32, 256])
let input2 = Tensor<Float> (ones: [1, 32, 32, 256])
let input3 = Tensor<Float> (zeros: [1, 32, 32, 256])
let input4 = Tensor<Float> (zeros: [1, 32, 32, 256])
let out = concatenate(input1, input2)
print(out.shape)

[1, 32, 32, 512]


In [ ]:
let out1 = input1.concatenated(with: input2, alongAxis: -1)
let out2 = input3.concatenated(with: input4, alongAxis: -1)
let out = out1.concatenated(with: out2, alongAxis: -1)
print(out.shape)

[1, 32, 32, 1024]


In [ ]:
public struct deepLabV3: Layer {
  @noDerivative let inputSize: Int

  public var convBlock1: ConvBlock
  public var convBlock2: ConvBlock
  public var residualBlock1: ConvBlock
  public var residualBlock2: ConvBlock
  public var residualBlock3: ConvBlock
  public var residualBlock4: ConvBlock
  public var convEntryBlock1: [DepthwiseConvBlock] = []
  public var convEntryBlock2: [DepthwiseConvBlock] = []
  public var convEntryBlock3: [DepthwiseConvBlock] = []
  public var convMiddleBlock: [MiddleFlow] = []
  public var convExitBlock1: [DepthwiseConvBlock] = []
  public var convExitBlock2: [DepthwiseConvBlock] = []
  public var aspp0: ConvBlock
  public var aspp1: ASPPLayer
  public var aspp2: ASPPLayer
  public var aspp3: ASPPLayer
  public var aspp4: ImageFeature
  public var featureProjection: ConvBlock
  public var concatProjection: ConvBlock
  public var decoderConv: [DepthwiseConvBlock] = []
  public var lastConv: ConvBlock

  public init(classCount: Int, inputSize: Int) {
    self.inputSize = inputSize

    // Entry flow
    convBlock1 = ConvBlock(filterShape: (3, 3, 3, 32), strides: (2,2))
    convBlock2 = ConvBlock(filterShape: (3, 3, 32, 64))
    
    // Block 1
    residualBlock1 = ConvBlock(filterShape: (1, 1, 64, 128), strides: (2,2), depthAct: false)
    convEntryBlock1.append(DepthwiseConvBlock(filterShape: (3, 3, 64, 128)))
    convEntryBlock1.append(DepthwiseConvBlock(filterShape: (3, 3, 128, 128)))
    convEntryBlock1.append(DepthwiseConvBlock(filterShape: (3, 3, 128, 128), strides: (2,2)))

    // Block 2
    residualBlock2 = ConvBlock(filterShape: (1, 1, 128, 256), strides: (2,2), depthAct: false)
    convEntryBlock2.append(DepthwiseConvBlock(filterShape: (3, 3, 128, 256)))
    convEntryBlock2.append(DepthwiseConvBlock(filterShape: (3, 3, 256, 256)))
    convEntryBlock2.append(DepthwiseConvBlock(filterShape: (3, 3, 256, 256), strides: (2,2)))

    // Block 3
    residualBlock3 = ConvBlock(filterShape: (1, 1, 256, 728), strides: (2,2), depthAct: false)
    convEntryBlock3.append(DepthwiseConvBlock(filterShape: (3, 3, 256, 728)))
    convEntryBlock3.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 728)))
    convEntryBlock3.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 728), strides: (2,2)))

    // Middle Flow
    convMiddleBlock = Array(repeating: MiddleFlow(), count: 16)
    
    // Exit Flow
    residualBlock4 = ConvBlock(filterShape: (1, 1, 728, 1024), strides: (1,1), depthAct: false)
    convExitBlock1.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 728), dilations: dilationRates.0))
    convExitBlock1.append(DepthwiseConvBlock(filterShape: (3, 3, 728, 1024), dilations: dilationRates.0))
    convExitBlock1.append(DepthwiseConvBlock(filterShape: (3, 3, 1024, 1024), dilations: dilationRates.0))

    convExitBlock2.append(DepthwiseConvBlock(
      filterShape: (3, 3, 1024, 1536), 
      dilations: dilationRates.1, 
      depthAct: true))
    convExitBlock2.append(DepthwiseConvBlock(
      filterShape: (3, 3, 1536, 1536), 
      dilations: dilationRates.1,
      depthAct: true))
    convExitBlock2.append(DepthwiseConvBlock(
      filterShape: (3, 3, 1536, 2048), 
      dilations: dilationRates.1,
      depthAct: true))

    // ASPP Module
    aspp0 = ConvBlock(filterShape: (1, 1, 2048, 256), epsilon: 1e-5)
    aspp1 = ASPPLayer(filterShape: (3, 3, 2048, 256), dilations: atrousRates.0)
    aspp2 = ASPPLayer(filterShape: (3, 3, 2048, 256), dilations: atrousRates.1)
    aspp3 = ASPPLayer(filterShape: (3, 3, 2048, 256), dilations: atrousRates.2)
    aspp4 = ImageFeature(filterShape: (1, 1, 2048, 256))
    concatProjection = ConvBlock(filterShape: (1, 1, 1280, 256), epsilon: 1e-5, applyDropout: true)

    // Decoder
    featureProjection = ConvBlock(filterShape: (1, 1, 256, 48), epsilon: 1e-5)
    decoderConv.append(DepthwiseConvBlock(
      filterShape: (3, 3, 304, 256), 
      epsilon: 1e-5,
      depthAct: true))
    decoderConv.append(DepthwiseConvBlock(
      filterShape: (3, 3, 256, 256), 
      epsilon: 1e-5,
      depthAct: true))

    // Last Conv Layer
    lastConv = ConvBlock(
      filterShape: (1, 1, 256, classCount),
      useBias: true,
      epsilon: 1e-5,
      applyBatchNorm: false,
      depthAct: false
    )
  }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
    var residual: Tensor<Float>
    var entryFlow = input.sequenced(through: convBlock1, convBlock2)
    
    // Block 1
    residual = residualBlock1(entryFlow)
    entryFlow = convEntryBlock1.differentiableReduce(entryFlow) {$1($0)}
    entryFlow = entryFlow + residual
    
    // Block 2
    residual = residualBlock2(entryFlow)
    entryFlow = entryFlow.sequenced(through: convEntryBlock2[0], convEntryBlock2[1])
    var lowLevelFeature = featureProjection(entryFlow)
    entryFlow = convEntryBlock2[2](entryFlow) + residual

    // Block 3
    residual = residualBlock3(entryFlow)
    entryFlow = convEntryBlock3.differentiableReduce(entryFlow) {$1($0)}
    entryFlow = entryFlow + residual
    
    // Middle Flow
    var middleFlow = entryFlow
    for idx in 0..<16 {
      residual = middleFlow
      middleFlow = convMiddleBlock[idx](middleFlow)
      middleFlow = middleFlow + residual
    }
    
    // Exit Flow
    var exitFlow = middleFlow
    residual = residualBlock4(exitFlow)
    exitFlow = convExitBlock1.differentiableReduce(exitFlow) {$1($0)}
    exitFlow = exitFlow + residual
    exitFlow = convExitBlock2.differentiableReduce(exitFlow) {$1($0)}

    // ASPP Module
    var asppModule: Tensor<Float>
    asppModule = aspp0(exitFlow).concatenated(with: aspp1(exitFlow), alongAxis: -1)
    asppModule = asppModule.concatenated(with: aspp2(exitFlow), alongAxis: -1)
    asppModule = asppModule.concatenated(with: aspp3(exitFlow), alongAxis: -1)
    asppModule = asppModule.concatenated(with: aspp4(exitFlow), alongAxis: -1)
    asppModule = concatProjection(asppModule)

    // Decoder
    var decoder = asppModule
    let hw = decoder.shape[1...2]
    decoder = resize(
      images: asppModule,
      size: (newHeight: 4 * hw[0], newWidth: 4 * hw[1]),
      method: .bilinear)
    decoder = decoder.concatenated(with: lowLevelFeature, alongAxis: -1)
    decoder = decoderConv.differentiableReduce(decoder) {$1($0)}
    decoder = resize(
      images: lastConv(decoder),
      size: (newHeight: self.inputSize, newWidth: self.inputSize),
      method: .bilinear)
    return decoder
  }
}

In [ ]:
let inputTensor = Tensor<Float>(ones: [1, inputSize, inputSize, 3])
var model = deepLabV3(classCount: classes, inputSize: inputSize)
var output = model(inputTensor)
print(output.shape)

[1, 512, 512, 21]


## Load Model Weights

In [ ]:
let modelName = "deeplabv3plus"
let backbone = "xception"

struct Weights {
    let name: String
    let layer: Tensor<Float>
}

func get_pretrained_weights_dict(modelName: String, backbone: String) -> Array<Weights> {
  var f = Python.None
  let weightFileName = "\(modelName)_\(backbone).npz"
  let weights = np.load(weightFileName)

  var weights_array = Array<Weights>()
  for param in weights {
      weights_array.append(Weights(name: String(param)!, layer: Tensor<Float>(numpy: weights[param])!))
  }
  return weights_array
}

var weightsDict = get_pretrained_weights_dict(modelName: modelName, backbone: backbone)

Fatal error: 'try!' expression unexpectedly raised an error: Python exception: Cannot load file containing pickled data when allow_pickle=False
Traceback:
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/npyio.py", line 457, in load
    raise ValueError("Cannot load file containing pickled data "
: file PythonKit/Python.swift, line 598
Current stack trace:
0    libswiftCore.so                    0x00007fb91cd46550 swift_reportError + 50
1    libswiftCore.so                    0x00007fb91cdb67f0 _swift_stdlib_reportFatalErrorInFile + 115
2    libswiftCore.so                    0x00007fb91ca4b072 <unavailable> + 1564786
3    libswiftCore.so                    0x00007fb91ca4ad80 <unavailable> + 1564032
4    libswiftCore.so                    0x00007fb91ca4b25b <unavailable> + 1565275
5    libswiftCore.so                    0x00007fb91ca49dd0 _assertionFailure(_:_:file:line:flags:) + 470
6    libswiftCore.so                    0x00007fb91caa23c0 swift_unexpectedError + 1100


: ignored

In [ ]:
let modelName = "deeplabv3plus"
let backbone = "xception"
let MODEL_DIR = "models"
let weightFileName = "\(modelName)_\(backbone).npz"

if ~os.path.exists(MODEL_DIR) == -1 {
  os.makedirs(MODEL_DIR)
}

let DATA_URL = "https://drive.google.com/file/d/1hCrK7u6ixkPfrlB0H5DfxOucGfkeyAhx/view?usp=sharing"
let path = tf.keras.utils.get_file(weightFileName, DATA_URL, cache_subdir:"", cache_dir: MODEL_DIR)
// let weights = np.load(path)

  65536/Unknown - 0s 1us/step

In [ ]:
var f = Python.True
print(~os.path.exists(MODEL_DIR))

-1


In [ ]:
print(path)

models/deeplabv3plus_xception.npz


In [ ]:
print(os.)

In [ ]:
subprocess.call("ls")

models	sample_data


Use `print()` to show values.


In [ ]:
let weights = np.load("deeplabv3plus_xception.npz", allow_pickle: Python.True)

Fatal error: 'try!' expression unexpectedly raised an error: Python exception: Failed to interpret file 'deeplabv3plus_xception.npz' as a pickle
Traceback:
  File "/usr/local/lib/python3.6/dist-packages/numpy/lib/npyio.py", line 463, in load
    "Failed to interpret file %s as a pickle" % repr(file))
: file PythonKit/Python.swift, line 608
Current stack trace:
0    libswiftCore.so                    0x00007fb91cd46550 swift_reportError + 50
1    libswiftCore.so                    0x00007fb91cdb67f0 _swift_stdlib_reportFatalErrorInFile + 115
2    libswiftCore.so                    0x00007fb91ca4b072 <unavailable> + 1564786
3    libswiftCore.so                    0x00007fb91ca4ad80 <unavailable> + 1564032
4    libswiftCore.so                    0x00007fb91ca4b25b <unavailable> + 1565275
5    libswiftCore.so                    0x00007fb91ca49dd0 _assertionFailure(_:_:file:line:flags:) + 470
6    libswiftCore.so                    0x00007fb91caa23c0 swift_unexpectedError + 1100


: ignored

In [ ]:
let command = "wget -nv -O- https://drive.google.com/file/d/1hCrK7u6ixkPfrlB0H5DfxOucGfkeyAhx/view?usp=sharing | tar xzf - -C ."
subprocess.call(command, shell: true)

In [ ]:
let weightFileName = "\(modelName)_\(backbone).npz"
let weights = np.load(weightFileName)
for params in weights {
  print(params)
}
let x = weightsDict.filter {key in return key.name.contains("aspp1_pointwise_kernel")}
print(x)